This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json
import statistics

In [3]:
# Define a function to create the url.
def quandl_url(first_date, last_date):
    """
    Creates a url to access Quandl json data on the stock with ticker name AFX_X at the Frankfurt Stock exchange between the 
    dates first_date and last_date
    """
    url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?'
    url += 'start_date=' + '\"' + first_date + '\"&'
    url += 'end_date=' + '\"' + last_date + '\"&'
    url += 'api_key=' + API_KEY
    return url

In [4]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

# The chosen start date and end date are the latest available days when this was written.
url = quandl_url('2020-11-06', '2020-11-06')
r = requests.get(url)
json_data = r.json()

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json_data)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2020-11-06', 'end_date': '2020-11-06', 'frequency': 'daily', 'data': [['2020-11-06', 115.2, 115.8, 114.8, 115.1, None, 190.0, 21831.0, None, None, None]], 'collapse': None, 'order': None}}


The following is exploration of the structure of the returned data.

In [6]:
print(type(json_data))

<class 'dict'>


In [7]:
print(json_data.keys())

dict_keys(['dataset_data'])


In [8]:
print(json_data['dataset_data'].keys())

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


In [9]:
print(type(json_data['dataset_data']['data']))

<class 'list'>


In [10]:
print(type(json_data['dataset_data']['data'][0]))

<class 'list'>


In [11]:
# Explore the types of data contained within the time series info.
# Spacing size is arbitrary for readability.
print('Index'.ljust(8,' ') + 'Column Name'.ljust(25, ' ') + 'Value'.ljust(13, ' ') + 'Type')
for idx, val in enumerate(json_data['dataset_data']['data'][0]):
    print(str(idx).ljust(8,' ') + json_data['dataset_data']['column_names'][idx].ljust(25, ' ') + str(val).ljust(13, ' ') + str(type(val)))

Index   Column Name              Value        Type
0       Date                     2020-11-06   <class 'str'>
1       Open                     115.2        <class 'float'>
2       High                     115.8        <class 'float'>
3       Low                      114.8        <class 'float'>
4       Close                    115.1        <class 'float'>
5       Change                   None         <class 'NoneType'>
6       Traded Volume            190.0        <class 'float'>
7       Turnover                 21831.0      <class 'float'>
8       Last Price of the Day    None         <class 'NoneType'>
9       Daily Traded Units       None         <class 'NoneType'>
10      Daily Turnover           None         <class 'NoneType'>


It appears that all of the data is referenced by the 'dataset_set' key. Nested within that, the stock time series information is a list of lists referenced by the key 'data'.

In [12]:
# Function to grab json information from the Quandl API.
# The particular database and ticker symbol are hardcoded for this project.
def get_stock_json(first_date, last_date):
    """
    Gets the Python dict obtained from the json form of the Quandl data for the stock with ticker name AFX_X 
    at the Frankfurt Stock exchange between the dates first_date and last_date
    """    
    url = quandl_url(first_date, last_date)
    r = requests.get(url)
    return r.json()

Analysis of the raw data shows that there are three suspicious entries. All three have opening prices of 'None' and all three are the same as the previous day's entries with minor rounding differences. These dates were April 13th, April 14th, and May 1st. Examination of https://www.tradinghours.com/exchanges/fsx/market-holidays/2017 shows that these days were holidays where the Exchange was closed. As a result, we intentionally omit any data where the opening price is listed as 'None'. This has mild effects on the mean and median, but doesn't change the other answers.

In [13]:
def create_stock_price_dict(json_data):
    """
    Converts the stock data json_data to a dictionary where keys are strings representing states and values are dictionaries
    with keys given by the column names in json_data and values by the corresponding date's entry.
    """
    column_names = json_data['dataset_data']['column_names']
    
    stock_price_dict = {}
    for stock_day in json_data['dataset_data']['data']:
        # Pair this day's data with the column names to form a dict for today's data.
        current_day_prices = dict(zip(column_names, stock_day))
        # Only use this data if the opening price is valid, as discussed above.
        if current_day_prices['Open'] is not None:
            # The date is used as a key in the top-level dictionary, so there's no need to repeat it.
            current_date = current_day_prices.pop('Date', None)
            stock_price_dict[current_date] = current_day_prices
        
    return stock_price_dict

In [14]:
# Combine the previous two functions to create a single function pipeline
def create_quandl_data(first_date, last_date):
    """
    Creates a dictionary from the json form of the Quandl data for the stock with ticker name AFX_X 
    at the Frankfurt Stock exchange between the dates first_date and last_date. This dictionary has format described by the
    function create_stock_dict
    """      
    stock_json = get_stock_json(first_date, last_date)
    return create_stock_price_dict(stock_json)

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

<b>Q1.</b> Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

<b>Q2.</b> Convert the returned JSON object into a Python dictionary.

In [15]:
# Both of these problems are handled at once by the pipeline set up above.
afx_2017_data = create_quandl_data('2017-01-01', '2017-12-31')
type(afx_2017_data)

dict

<b>Q3.</b> Calculate what the highest and lowest opening prices were for the stock in this period.

In [16]:
# Collect the opening prices
opening_prices = [ daily_prices['Open'] for daily_prices in afx_2017_data.values()]

print('The highest opening price was {:.2f}'.format(max(opening_prices)))
print('The lowest opening price was {:.2f}'.format(min(opening_prices)))

The highest opening price was 53.11
The lowest opening price was 34.00


<b>Q4.</b> What was the largest change in any one day (based on High and Low price)?

In [17]:
# Collection High price less Low price
price_differences = [ daily_prices['High'] - daily_prices['Low'] for daily_prices in afx_2017_data.values() ]
        
# Quick validation, to make sure that high_price >= low_price for all days
print(min(price_differences) >= 0)

True


In [18]:
# The requested value is then the max of this collection.
print('The maximum daily change in price was {:.2f}'.format(max(price_differences)))

The maximum daily change in price was 2.81


<b>Q5.</b> What was the largest change between any two days (based on Closing Price)?

In [19]:
# Extract the closing prices
closing_prices = [ daily_prices['Close'] for daily_prices in afx_2017_data.values()]

# We subtract the sublist of the last N-1 entries from the sublist of the first N-1 entries where N is the list length
# This gives the day-to-day change. We are interested in the largest magnitude, so we track the absolute value of the changes.
closing_differences = [ abs(a - b) for a, b in zip(closing_prices[1:], closing_prices[:-1])]

# Validate the computations by checking to verify no negative values
print(min(closing_differences) >= 0)

True


In [20]:
print('The maximum day-to-day change in price was {:.2f}'.format(max(closing_differences)))

The maximum day-to-day change in price was 2.56


<b>Q6.</b> What was the average daily trading volume during this year?

In [21]:
# Extract the daily volumes
daily_volumes = [ daily_prices['Traded Volume'] for daily_prices in afx_2017_data.values()]

In [22]:
mean_volume = sum(daily_volumes) / len(daily_volumes)
print('The mean daily volume was {:,.2f} shares'.format(mean_volume))

The mean daily volume was 89,140.98 shares


This agrees with the `mean` function provided in the `statistics` package.

In [23]:
print(statistics.mean(daily_volumes) == mean_volume)

True


<b>Q7.</b> (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

It is possible to use the `median` function from the `statistics` package.

In [24]:
print('The median daily volume was {:,.2f} shares'.format(statistics.median(daily_volumes)))

The median daily volume was 74,723.50 shares


Based on problem wording, we'll also implement our own median function to verify.

In [25]:
def median(x):
    """
    Determines the median of a list of numeric values.
    """
    sorted_list = x
    sorted_list.sort()
    l = len(sorted_list)
    if (l % 2 == 1):
        # If there are an odd number of list entries, return the middle one.
        i = int((l - 1) / 2)
        return sorted_list[i]
    else:
         # Otherwise, return the mean of the middle two entries.
        i = int(l / 2)
        return (sorted_list[i - 1] + sorted_list[i]) / 2

In [26]:
print('The median daily volume was {:,.2f} shares'.format(median(daily_volumes)))

The median daily volume was 74,723.50 shares


The two values match, as required.

In [27]:
print(statistics.median(daily_volumes) == median(daily_volumes))

True
